# 0909

### Kaggle 데이터 내려받기
1. 캐글 API Token 발급
- 캐글 사이트 account 페이지 이동 > API 섹션 내 CREATE NEW API TOKEN 클릭 > 다운로드된 파일 kaggle.json 확인

https://www.kaggle.com/settings

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rodneykim","key":"864372c8d1d22e59231f822167390d90"}'}

In [3]:
!ls


drive  kaggle.json  sample_data


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!ls ~/.kaggle/ -la

total 16
drwxr-xr-x 2 root root 4096 Sep  9 02:43 .
drwx------ 1 root root 4096 Sep  9 02:43 ..
-rw------- 1 root root   65 Sep  9 02:43 kaggle.json


In [6]:
!pip install --upgrade kaggle

권한을 세 자리로 나타내는 방식:
- 첫 번째 숫자 (6): **파일 소유자(owner)**에 대한 권한을 나타냅니다.
- 두 번째 숫자 (0): **그룹(group)**에 대한 권한을 나타냅니다.
- 세 번째 숫자 (0): **기타 사용자(others)**에 대한 권한을 나타냅니다.

각 숫자는 다음과 같이 구성됩니다:
- 4: 읽기 권한 (read)
- 2: 쓰기 권한 (write)
- 1: 실행 권한 (execute)
- 0: 아무 권한 없음

In [7]:
!kaggle --version

Kaggle API 1.6.17


In [8]:
!kaggle datasets download -d biaiscience/dogs-vs-cats


Dataset URL: https://www.kaggle.com/datasets/biaiscience/dogs-vs-cats
License(s): DbCL-1.0
100% 814M/817M [00:35<00:00, 27.1MB/s]
100% 817M/817M [00:35<00:00, 23.9MB/s]


In [11]:
!ls


dogs-vs-cats.zip  drive  kaggle.json  sample_data  test  train


In [10]:
!unzip -qq dogs-vs-cats.zip

In [12]:
# mv는 파일이나 디렉토리를 이동하거나 이름을 변경할때 사용
!mv train train1

In [14]:
!mv test test1

In [15]:
!ls

dogs-vs-cats.zip  drive  kaggle.json  sample_data  test1  train1


In [16]:
!mv /content/train1/train /content/
!mv /content/test1/test /content/

In [17]:
!ls

dogs-vs-cats.zip  drive  kaggle.json  sample_data  test  test1	train  train1


In [18]:
!rm -rf test1 train1

In [19]:
!ls

dogs-vs-cats.zip  drive  kaggle.json  sample_data  test  train


In [27]:
import os, shutil, pathlib

original_dir = pathlib.Path('train')
new_base_dir = pathlib.Path('cats_vs_dogs_small')

def make_subset(subset_name, start_index, end_index):
    for category in ('cat', 'dog'):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f'{category}.{i}.jpg' for i in range(start_index, end_index)]
        for fname in fnames:
            src = original_dir / fname
            dst = dir / fname
            shutil.copyfile(src, dst)

make_subset('train', 0, 1000)
make_subset('validation', 1000, 1500)
make_subset('test', 1500, 2000)

In [28]:
!ls cats_vs_dogs_small/

test  train  validation


In [29]:
!ls cats_vs_dogs_small/train/cat

cat.0.jpg    cat.229.jpg  cat.358.jpg  cat.487.jpg  cat.615.jpg  cat.744.jpg  cat.873.jpg
cat.100.jpg  cat.22.jpg   cat.359.jpg  cat.488.jpg  cat.616.jpg  cat.745.jpg  cat.874.jpg
cat.101.jpg  cat.230.jpg  cat.35.jpg   cat.489.jpg  cat.617.jpg  cat.746.jpg  cat.875.jpg
cat.102.jpg  cat.231.jpg  cat.360.jpg  cat.48.jpg   cat.618.jpg  cat.747.jpg  cat.876.jpg
cat.103.jpg  cat.232.jpg  cat.361.jpg  cat.490.jpg  cat.619.jpg  cat.748.jpg  cat.877.jpg
cat.104.jpg  cat.233.jpg  cat.362.jpg  cat.491.jpg  cat.61.jpg	 cat.749.jpg  cat.878.jpg
cat.105.jpg  cat.234.jpg  cat.363.jpg  cat.492.jpg  cat.620.jpg  cat.74.jpg   cat.879.jpg
cat.106.jpg  cat.235.jpg  cat.364.jpg  cat.493.jpg  cat.621.jpg  cat.750.jpg  cat.87.jpg
cat.107.jpg  cat.236.jpg  cat.365.jpg  cat.494.jpg  cat.622.jpg  cat.751.jpg  cat.880.jpg
cat.108.jpg  cat.237.jpg  cat.366.jpg  cat.495.jpg  cat.623.jpg  cat.752.jpg  cat.881.jpg
cat.109.jpg  cat.238.jpg  cat.367.jpg  cat.496.jpg  cat.624.jpg  cat.753.jpg  cat.882.jpg
cat.10.jpg  

In [30]:
!ls

cats_vs_dogs_small  dogs-vs-cats.zip  drive  kaggle.json  sample_data  test  train


In [31]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / 'train',
    image_size=(180, 180),
    batch_size=32
)

validation_dataset = image_dataset_from_directory(
    new_base_dir / 'validation',
    image_size=(180, 180),
    batch_size=32
)
test_dataset = image_dataset_from_directory(
    new_base_dir / 'test',
    image_size=(180, 180),
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [32]:
for data_batch, labels_batch in train_dataset:
    print("데이터 배치 크기: ",data_batch.shape)
    print("레이블 배치 크기: ",labels_batch.shape)
    break

데이터 배치 크기:  (32, 180, 180, 3)
레이블 배치 크기:  (32,)


In [35]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

In [37]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [38]:
import tensorflow as tf
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='convnet_from_scratch.keras',
        save_best_only=True,
        monitor='val_loss')
]

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,
    callbacks=callbacks
)


Epoch 1/30
15/63 ━━━━━━━━━━━━━━━━━━━━ 2:56 4s/step - accuracy: 0.4231 - loss: 0.7268

KeyboardInterrupt: 